<a href="https://colab.research.google.com/github/AbdulHaqq2001/Database_management/blob/main/Week11_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

# Create a SQLite database in memory (you can change this to a file path if you want to persist the database)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the 'student' table
cursor.execute('''
    CREATE TABLE student (
        id INTEGER PRIMARY KEY,
        name TEXT,
        tot_cred INTEGER
    )
''')

# Insert sample data into the 'student' table
sample_data = [
    ('Alice', 25),
    ('Bob', 45),
    ('Charlie', 70),
    ('David', 100),
    ('Eva', 130),
]

cursor.executemany('INSERT INTO student (name, tot_cred) VALUES (?, ?)', sample_data)

# Query the status of each student based on total credits
cursor.execute('''
    SELECT
        name,
        tot_cred,
        CASE
            WHEN tot_cred < 30 THEN 'Freshman'
            WHEN tot_cred >= 30 AND tot_cred < 60 THEN 'Sophomore'
            WHEN tot_cred >= 60 AND tot_cred <= 90 THEN 'Junior'
            WHEN tot_cred >= 90 AND tot_cred <= 120 THEN 'Senior'
            WHEN tot_cred > 120 THEN 'Graduate'
        END AS status
    FROM
        student
''')

# Fetch and print the results
results = cursor.fetchall()
for row in results:
    print(f"{row[0]} - Total Credits: {row[1]}, Status: {row[2]}")

# Close the connection
conn.close()


Alice - Total Credits: 25, Status: Freshman
Bob - Total Credits: 45, Status: Sophomore
Charlie - Total Credits: 70, Status: Junior
David - Total Credits: 100, Status: Senior
Eva - Total Credits: 130, Status: Graduate


In [2]:
import sqlite3

# Create a SQLite database in memory (you can change this to a file path if you want to persist the database)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the 'student' table
cursor.execute('''
    CREATE TABLE student (
        id INTEGER PRIMARY KEY,
        name TEXT,
        dept_name TEXT,
        tot_cred INTEGER
    )
''')

# Insert sample data into the 'student' table
sample_data = [
    (1, 'Alice', 'Computer Science', 25),
    (2, 'Bob', 'Mathematics', 45),
    (3, 'Charlie', 'Physics', 70),
    (4, 'David', 'Biology', 100),
    (5, 'Eva', 'Chemistry', 130),
]

cursor.executemany('INSERT INTO student (id, name, dept_name, tot_cred) VALUES (?, ?, ?, ?)', sample_data)

# Query the number of students with each status
cursor.execute('''
    SELECT
        CASE
            WHEN tot_cred < 30 THEN 'Freshman'
            WHEN tot_cred >= 30 AND tot_cred < 60 THEN 'Sophomore'
            WHEN tot_cred >= 60 AND tot_cred < 90 THEN 'Junior'
            WHEN tot_cred >= 90 AND tot_cred < 120 THEN 'Senior'
            WHEN tot_cred >= 120 THEN 'Graduate'
        END AS status,
        COUNT(*) AS num_students
    FROM
        student
    GROUP BY
        status
''')

# Fetch and print the results
results = cursor.fetchall()
for row in results:
    print(f"Status: {row[0]}, Number of Students: {row[1]}")

# Close the connection
conn.close()


Status: Freshman, Number of Students: 1
Status: Graduate, Number of Students: 1
Status: Junior, Number of Students: 1
Status: Senior, Number of Students: 1
Status: Sophomore, Number of Students: 1


In [3]:
import sqlite3

# Create a SQLite database in memory (you can change this to a file path if you want to persist the database)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the 'courses' table
cursor.execute('''
    CREATE TABLE courses (
        student_id INTEGER,
        course_name TEXT,
        grade TEXT,
        credits INTEGER
    )
''')

# Insert sample data into the 'courses' table
sample_data = [
    (1, 'Math', 'A', 3),
    (1, 'Physics', 'B+', 4),
    (2, 'Chemistry', 'A+', 3),
    (2, 'Biology', 'B', 4),
    (3, 'History', 'C', 3),
    (3, 'English', 'A', 3),
    # Add more sample data as needed
]

cursor.executemany('INSERT INTO courses (student_id, course_name, grade, credits) VALUES (?, ?, ?, ?)', sample_data)

# Question 1: Calculate the GPA for each student who has taken any course in 2008, Spring semester
cursor.execute('''
    SELECT
        student_id,
        SUM(CASE
            WHEN grade = 'A+' THEN 4.50 * credits
            WHEN grade = 'A' THEN 4.00 * credits
            WHEN grade = 'B+' THEN 3.50 * credits
            WHEN grade = 'B' THEN 3.00 * credits
            WHEN grade = 'C+' THEN 2.50 * credits
            WHEN grade = 'C' THEN 2.00 * credits
            WHEN grade = 'D+' THEN 1.50 * credits
            WHEN grade = 'D' THEN 1.00 * credits
            ELSE 0
        END) / SUM(4.50 * credits) AS gpa
    FROM
        courses
    GROUP BY
        student_id
''')

# Fetch and print the results for Question 1
gpa_results = cursor.fetchall()
for row in gpa_results:
    print(f"Student ID: {row[0]}, GPA: {row[1]}")

# Question 2: Find the number of students according to category
cursor.execute('''
    SELECT
        CASE
            WHEN gpa BETWEEN 3.75 AND 4.50 THEN 'A'
            WHEN gpa BETWEEN 3.00 AND 3.75 THEN 'B'
            WHEN gpa BETWEEN 2.00 AND 3.00 THEN 'C'
            WHEN gpa BETWEEN 1.50 AND 2.00 THEN 'D'
            WHEN gpa < 1.50 THEN 'F'
        END AS category,
        COUNT(*) AS num_students
    FROM (
        SELECT
            student_id,
            SUM(CASE
                WHEN grade = 'A+' THEN 4.50 * credits
                WHEN grade = 'A' THEN 4.00 * credits
                WHEN grade = 'B+' THEN 3.50 * credits
                WHEN grade = 'B' THEN 3.00 * credits
                WHEN grade = 'C+' THEN 2.50 * credits
                WHEN grade = 'C' THEN 2.00 * credits
                WHEN grade = 'D+' THEN 1.50 * credits
                WHEN grade = 'D' THEN 1.00 * credits
                ELSE 0
            END) / SUM(4.50 * credits) AS gpa
        FROM
            courses
        GROUP BY
            student_id
    ) AS student_gpa
    GROUP BY
        category
''')

# Fetch and print the results for Question 2
category_results = cursor.fetchall()
for row in category_results:
    print(f"Category: {row[0]}, Number of Students: {row[1]}")

# Close the connection
conn.close()


Student ID: 1, GPA: 0.8253968253968254
Student ID: 2, GPA: 0.8095238095238095
Student ID: 3, GPA: 0.6666666666666666
Category: F, Number of Students: 3
